# Reading events from RGTDB

Query to get JSON:

https://rgtdb.com/events/json?search=&offset=0&limit=100

This reads upcoming events from rgtdb.com and converts them into iCAL format and writes to a file for manual import. Then this saves or updates the events in a Google calendar.

## Get data and cache response

In [45]:
import pandas as pd 

import datetime
from datetime import datetime as dt

import io
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Debug level logging turned on")

import requests
from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter

sess = requests.session()
cached_sess = CacheControl(sess, cache = FileCache('.web_cache'), heuristic=ExpiresAfter(hours=1))

try:
    response = cached_sess.get('https://rgtdb.com/events/json?search=&offset=0&limit=200') # Get 200 events. Should be about a week's worth of events
    response.raise_for_status()

except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

logger.setLevel(logging.ERROR)

print(dt.now())

DEBUG:root:Debug level logging turned on
DEBUG:cachecontrol.controller:Looking up "https://rgtdb.com/events/json?search=&offset=0&limit=200" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 15134
DEBUG:cachecontrol.controller:Freshness lifetime from expires: 3601
DEBUG:cachecontrol.controller:The cached response is "stale" with no etag, purging
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rgtdb.com:443
DEBUG:urllib3.connectionpool:https://rgtdb.com:443 "GET /events/json?search=&offset=0&limit=200 HTTP/1.1" 200 None
DEBUG:cachecontrol.controller:Updating cache with response from "https://rgtdb.com/events/json?search=&offset=0&limit=200"
DEBUG:cachecontrol.controller:Caching b/c of expires header
2021-02-28 13:24:30.510362


In [46]:
response.json().keys()

dict_keys(['total', 'rows'])

## Convert JSON to Pandas Dataframe

Not necessary, but hey, pretending to be a data scientist feels cool.

In [47]:
df = pd.json_normalize(response.json(), 'rows')
df

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
0,MRETT 15,02-27 19:00,/events/72716,[race],1,27.89 km,158 m,158 m,"MRETT15 - Aswan, Egypt",/courses/144175,False
1,Sunday Social,02-28 10:00,/events/59876,[groupride],23,35.31 km,450 m,450 m,Dirty Reiver,/courses/80194,False
2,High Peak Road Race,02-28 10:00,/events/67486,[race],45,80.10 km,1.17 km,1.17 km,High Peak,/courses/103944,False
3,WKG's Watts Occurring,02-28 11:00,/events/70848,[groupride],23,51.29 km,194 m,194 m,Padrones Ride,/courses/95596,False
4,Breakfast Club,02-28 12:00,/events/59722,[groupride],11,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False
...,...,...,...,...,...,...,...,...,...,...,...
195,#100 Five Peaks,03-25 19:30,/events/62608,[groupride],1,38.28 km,948 m,881 m,#100 5Peak Challenge,/courses/110793,False
196,#100 Salcombe Hill,03-26 12:30,/events/62634,[groupride],1,5.68 km,198 m,84 m,#100 Salcombe Hill,/courses/110785,False
197,R2G Friday Night Crit,03-26 18:30,/events/70078,[race],3,22.07 km,106 m,106 m,Pdq Odd Down Circuit,/courses/107189,True
198,#100 Combe Gibbet,03-26 19:30,/events/62635,[groupride],1,5.50 km,118 m,176 m,#100 Combe Gibbet,/courses/110781,False


## Convert start time into datetime format - and guess at the year

This will be an issue every year around new year.

In [48]:

this_year = str(dt.today().year)
df['date'] = pd.to_datetime(df['startAt'] + ' ' + this_year, format='%m-%d %H:%M %Y', utc=True)
df.set_index('date', inplace=True)


In [49]:
df.head()

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
date,,,,,,,,,,,
2021-02-27 19:00:00+00:00,MRETT 15,02-27 19:00,/events/72716,[race],1,27.89 km,158 m,158 m,"MRETT15 - Aswan, Egypt",/courses/144175,False
2021-02-28 10:00:00+00:00,Sunday Social,02-28 10:00,/events/59876,[groupride],23,35.31 km,450 m,450 m,Dirty Reiver,/courses/80194,False
2021-02-28 10:00:00+00:00,High Peak Road Race,02-28 10:00,/events/67486,[race],45,80.10 km,1.17 km,1.17 km,High Peak,/courses/103944,False
2021-02-28 11:00:00+00:00,WKG's Watts Occurring,02-28 11:00,/events/70848,[groupride],23,51.29 km,194 m,194 m,Padrones Ride,/courses/95596,False
2021-02-28 12:00:00+00:00,Breakfast Club,02-28 12:00,/events/59722,[groupride],11,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False


## Use icalendar package to create ICAL format events

* GitHub: https://github.com/collective/icalendar


In [50]:
from datetime import timedelta
from icalendar import vCalAddress, vText
from icalendar import Calendar, Event
import pytz

cal = Calendar()
cal.add('prodid', '-//My calendar product//mxm.com//')
cal.add('version', '2.0')

for index, row in df.iterrows():
    print(index, row['name'], row['tags'])
    event = Event()
    event['uid'] = row['detailsUrl']
    event.add('summary', str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']))
    event.add('dtstart', index)
    event.add('dtend', index  + timedelta(hours=1))
    event.add('url', 'https://rgtdb.com' + row['detailsUrl'])
    event.add('description', row['distance'] + ' ' + 'https://rgtdb.com' + row['detailsUrl'])
    event.add('color', 'Tomato')
    event['location'] = vText(row['roadName'])

    cal.add_component(event)

2021-02-27 19:00:00+00:00 MRETT 15 ['race']
2021-02-28 10:00:00+00:00 Sunday Social ['groupride']
2021-02-28 10:00:00+00:00 High Peak Road Race ['race']
2021-02-28 11:00:00+00:00 WKG's Watts Occurring ['groupride']
2021-02-28 12:00:00+00:00 Breakfast Club ['groupride']
2021-02-28 13:30:00+00:00 MRETT14 Siboire CCS ['race']
2021-02-28 14:00:00+00:00 Weekend Warrior ['groupride']
2021-02-28 14:00:00+00:00 GFNS JENSIE E-FONDO ['race']
2021-02-28 14:00:00+00:00 MRETT14 Lou's Team ['race']
2021-02-28 14:00:00+00:00 PDQ crits south  ['race']
2021-02-28 14:30:00+00:00 OTR WOMEN'WEEKENDER ['race']
2021-02-28 15:00:00+00:00 Lou's Sunday Group Ride ['groupride']
2021-02-28 15:00:00+00:00 Sunday Social ['groupride']
2021-02-28 15:00:00+00:00 Giant of Provence ['race']
2021-02-28 15:00:00+00:00 GFNS JENSIE E-FONDO ['race']
2021-02-28 15:00:00+00:00 Napoleon Dolomite ['race']
2021-02-28 15:30:00+00:00 OTR Spanish For Improvers ['race']
2021-02-28 15:45:00+00:00 MRETT14-Westerley ['race']
2021-02-28

## Write to File

Can use to manually import into Google, other calendars

In [51]:
import tempfile, os
f = open('./rgt_events.ics', 'wb')
f.write(cal.to_ical())
f.close()

## Use gcsa for Simplified Access to Google Calendar API

* GitHub: https://github.com/kuzmoyev/google-calendar-simple-api
* Docs: https://google-calendar-simple-api.readthedocs.io/en/latest/index.html

Need to add socket timeout of 5 minutes due to slow response on my machine

### Shared Calendar

* Calendar ID: 3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com
* Google sharing link: https://calendar.google.com/calendar/u/0?cid=M2U4Z2F1OGJvbW1mamszM2o5MnJ2NWs3cTBAZ3JvdXAuY2FsZW5kYXIuZ29vZ2xlLmNvbQ
* ICS format for e.g., Apple Calendar: https://calendar.google.com/calendar/ical/3e8gau8bommfjk33j92rv5k7q0%40group.calendar.google.com/public/basic.ics


In [52]:
from gcsa.event import Event as gcEvent
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import socket
socket.setdefaulttimeout(300) # 5 minutes

EMAIL_FOR_CAL = '3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com'

calendar = GoogleCalendar(EMAIL_FOR_CAL)


## Cleanup: Delete All Events from Google Calendar

Uncomment to clean up calendar.

In [53]:
#calendar.clear() # This gives an error in the Google API

#for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):
#    print('Deleting:', event, event.event_id)
#    calendar.delete_event(event)

## Find existing events, mark for update instead of creation

In [54]:
import re

reExtractName = re.compile(" \[\'.*")

df['cal_id'] = None
df['event_obj'] = None
df['found'] = False

for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):

    print(event)

    rideName = reExtractName.sub("", event.summary) # Get substring from event summary with just the name

    df.loc[(df['name'] == rideName) & (df.index == event.start), ['found', 'cal_id', 'event_obj']] = [True, event.event_id, event]


 28 ['groupride'] 1/35.02 km/1.06 km
2021-02-27 14:00:00+00:00 - Wacky Races 31 ['groupride'] 1/35.10 km/171 m
2021-02-27 14:30:00+00:00 - Wacky Races 32 ['groupride'] 1/33.75 km/197 m
2021-02-27 15:00:00+00:00 - Wacky Races 33 ['groupride'] 1/32.78 km/191 m
2021-02-27 15:30:00+00:00 - Wacky Races 34 ['groupride'] 1/34.97 km/277 m
2021-02-27 17:00:00+00:00 - Wacky Races 37 ['groupride'] 1/28.89 km/312 m
2021-02-27 17:30:00+00:00 - Wacky Races 38 ['groupride'] 1/35.31 km/335 m
2021-02-27 18:30:00+00:00 - Wacky Races 40 ['groupride'] 1/31.52 km/361 m
2021-02-27 19:00:00+00:00 - Wacky Races 41 ['groupride'] 1/30.55 km/386 m
2021-02-27 19:00:00+00:00 - Weekend Warrior ['groupride'] 1/22.43 km/515 m
2021-02-27 19:30:00+00:00 - Wacky Races 42 ['groupride'] 1/53.76 km/393 m
2021-02-27 20:00:00+00:00 - Wacky Races 43 ['groupride'] 1/31.29 km/489 m
2021-02-27 20:30:00+00:00 - Wacky Races 44 ['groupride'] 1/34.88 km/498 m
2021-02-27 21:00:00+00:00 - Wacky Races 45 ['groupride'] 1/37.82 km/515 m


In [55]:
df.loc[df['found'] == True]


,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-02-27 19:00:00+00:00,MRETT 15,02-27 19:00,/events/72716,[race],1,27.89 km,158 m,158 m,"MRETT15 - Aswan, Egypt",/courses/144175,False,224fjnfuns7s3etc6clgbpdlik,2021-02-27 19:00:00+00:00 - MRETT 15 ['race'] ...,True
2021-02-28 10:00:00+00:00,Sunday Social,02-28 10:00,/events/59876,[groupride],23,35.31 km,450 m,450 m,Dirty Reiver,/courses/80194,False,2387p9ej3bmjspbh35bfhmn44g,2021-02-28 10:00:00+00:00 - Sunday Social ['gr...,True
2021-02-28 10:00:00+00:00,High Peak Road Race,02-28 10:00,/events/67486,[race],45,80.10 km,1.17 km,1.17 km,High Peak,/courses/103944,False,l6b693rrlubv689ctng9vgo3gk,2021-02-28 10:00:00+00:00 - High Peak Road Rac...,True
2021-02-28 11:00:00+00:00,WKG's Watts Occurring,02-28 11:00,/events/70848,[groupride],23,51.29 km,194 m,194 m,Padrones Ride,/courses/95596,False,p4f6i6lko7apqttp35pbbsia74,2021-02-28 11:00:00+00:00 - WKG's Watts Occurr...,True
2021-02-28 12:00:00+00:00,Breakfast Club,02-28 12:00,/events/59722,[groupride],11,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False,d7inckh2273n1q1dq99od5vonc,2021-02-28 12:00:00+00:00 - Breakfast Club ['g...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-24 19:00:00+00:00,Wingman Cup - Stage 7,03-24 19:00,/events/49516,[race],18,24.04 km,699 m,262 m,Jay Peak,/courses/89054,False,6gc69gf5k9mp59k7bc45koli38,2021-03-24 19:00:00+00:00 - Wingman Cup - Stag...,True
2021-03-24 19:30:00+00:00,#100 Dartmoor,03-24 19:30,/events/62601,[groupride],1,28.70 km,848 m,727 m,#100 Dartmoor,/courses/110801,False,ut0gg5sphls2ivlgfvutl0a224,2021-03-24 19:30:00+00:00 - #100 Dartmoor ['gr...,True
2021-03-25 12:30:00+00:00,#100 Rundlestone,03-25 12:30,/events/62604,[groupride],1,12.84 km,433 m,124 m,#100 Rundlestone,/courses/110797,False,i8qnjnov7m71rulg4deslejd90,2021-03-25 12:30:00+00:00 - #100 Rundlestone [...,True


In [56]:
df.loc[df['found'] == False]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-03-07 16:00:00+00:00,The “Boomerang” 10m TT,03-07 16:00,/events/72861,"[race, itt]",6,16.16 km,109 m,134 m,P164 10m TT Course,/courses/126685,False,None,None,False
2021-03-25 19:30:00+00:00,#100 Five Peaks,03-25 19:30,/events/62608,[groupride],1,38.28 km,948 m,881 m,#100 5Peak Challenge,/courses/110793,False,None,None,False
2021-03-26 12:30:00+00:00,#100 Salcombe Hill,03-26 12:30,/events/62634,[groupride],1,5.68 km,198 m,84 m,#100 Salcombe Hill,/courses/110785,False,None,None,False
2021-03-26 18:30:00+00:00,R2G Friday Night Crit,03-26 18:30,/events/70078,[race],3,22.07 km,106 m,106 m,Pdq Odd Down Circuit,/courses/107189,True,None,None,False
2021-03-26 19:30:00+00:00,#100 Combe Gibbet,03-26 19:30,/events/62635,[groupride],1,5.50 km,118 m,176 m,#100 Combe Gibbet,/courses/110781,False,None,None,False
2021-03-27 06:00:00+00:00,WKG's SDW. Stage 3,03-27 06:00,/events/61493,[race],1,26.96 km,635 m,469 m,SDW Amberley To Devi,/courses/107881,False,None,None,False


## Add All Events to Google Calendar

In [57]:
from gcsa.event import Event as gcEvent

for index, row in df.iterrows():

    evntSummary = str(str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']) + '/' + row['distance'] + '/' + row['elevationGain'])

    evntDescription = 'Signups: ' +  str(row['signUps']) + '\n' + 'Distance: ' + row['distance'] + '\n' +  'Elevation gain: ' + row['elevationGain'] + '\n' + 'Descent: ' + row['elevationLost'] + '\n' + 'https://rgtdb.com' + str(row['detailsUrl'])

    evntString = row['detailsUrl'].replace('/events/', '')

    if row['found'] == False:

        print('+New event: ', index, row['name'], row['tags'], evntString)

        evntColor = '1'

        if "groupride" in row['tags']:
            evntColor = '2'
        elif 'pro' in row['tags']:
            evntColor = '3'
        elif 'elimination' in row['tags']:
            evntColor = '4'
        elif "itt" in row['tags']:
            evntColor = '5'
        elif "race" in row['tags']:
            evntColor = '6'

        event = gcEvent(
            evntSummary,
            start=index,
            timezone='UTC',
            location=str(row['roadName']),
            description=evntDescription,
            event_id=evntString,
            color = evntColor
        )

        print('ID before add:', event.event_id)
        ret_event = calendar.add_event(event)
        print('ID after add:', event.event_id, 'Returned event ID:', ret_event.event_id)

    else:

        print('-Updating event: ', index, row['name'], row['tags'], evntString)

        event = row['event_obj']

        event.summary = evntSummary
        event.description = evntDescription

        calendar.update_event(event)

-Updating event:  2021-02-27 19:00:00+00:00 MRETT 15 ['race'] 72716
-Updating event:  2021-02-28 10:00:00+00:00 Sunday Social ['groupride'] 59876
-Updating event:  2021-02-28 10:00:00+00:00 High Peak Road Race ['race'] 67486
-Updating event:  2021-02-28 11:00:00+00:00 WKG's Watts Occurring ['groupride'] 70848
-Updating event:  2021-02-28 12:00:00+00:00 Breakfast Club ['groupride'] 59722
-Updating event:  2021-02-28 13:30:00+00:00 MRETT14 Siboire CCS ['race'] 71026
-Updating event:  2021-02-28 14:00:00+00:00 Weekend Warrior ['groupride'] 59849
-Updating event:  2021-02-28 14:00:00+00:00 GFNS JENSIE E-FONDO ['race'] 68036
-Updating event:  2021-02-28 14:00:00+00:00 MRETT14 Lou's Team ['race'] 68172
-Updating event:  2021-02-28 14:00:00+00:00 PDQ crits south  ['race'] 69035
-Updating event:  2021-02-28 14:30:00+00:00 OTR WOMEN'WEEKENDER ['race'] 71219
-Updating event:  2021-02-28 15:00:00+00:00 Lou's Sunday Group Ride ['groupride'] 71289
-Updating event:  2021-02-28 15:00:00+00:00 Sunday 

In [58]:
df.loc[df['signUps'] > 9]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-02-28 10:00:00+00:00,Sunday Social,02-28 10:00,/events/59876,[groupride],23,35.31 km,450 m,450 m,Dirty Reiver,/courses/80194,False,2387p9ej3bmjspbh35bfhmn44g,2021-02-28 10:00:00+00:00 - Sunday Social ['gr...,True
2021-02-28 10:00:00+00:00,High Peak Road Race,02-28 10:00,/events/67486,[race],45,80.10 km,1.17 km,1.17 km,High Peak,/courses/103944,False,l6b693rrlubv689ctng9vgo3gk,2021-02-28 10:00:00+00:00 - High Peak Road Rac...,True
2021-02-28 11:00:00+00:00,WKG's Watts Occurring,02-28 11:00,/events/70848,[groupride],23,51.29 km,194 m,194 m,Padrones Ride,/courses/95596,False,p4f6i6lko7apqttp35pbbsia74,2021-02-28 11:00:00+00:00 - WKG's Watts Occurr...,True
2021-02-28 12:00:00+00:00,Breakfast Club,02-28 12:00,/events/59722,[groupride],11,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False,d7inckh2273n1q1dq99od5vonc,2021-02-28 12:00:00+00:00 - Breakfast Club ['g...,True
2021-02-28 14:00:00+00:00,GFNS JENSIE E-FONDO,02-28 14:00,/events/68036,[race],51,43.43 km,1.30 km,616 m,7. E Fondo Jensi V3 Test,/courses/107793,False,e7sakeo8ikpsghr4eq8ddbbqkk,2021-02-28 14:00:00+00:00 - GFNS JENSIE E-FOND...,True
2021-02-28 14:00:00+00:00,PDQ crits south,02-28 14:00,/events/69035,[race],12,44.13 km,234 m,234 m,Colerne,/courses/119013,True,lb6v2r65gec6cvcnljk7a1nptc,2021-02-28 14:00:00+00:00 - PDQ crits south [...,True
2021-02-28 15:00:00+00:00,Lou's Sunday Group Ride,02-28 15:00,/events/71289,[groupride],36,38.22 km,148 m,148 m,Kielder,/courses/47238,False,diok3fo3qmf82s9mpnekjq9qn4,2021-02-28 15:00:00+00:00 - Lou's Sunday Group...,True
2021-02-28 15:00:00+00:00,GFNS JENSIE E-FONDO,02-28 15:00,/events/56539,[race],145,43.43 km,1.30 km,616 m,7. E Fondo Jensi V3 Test,/courses/107793,False,m0n2m11kue408vkatbsourkgu4,2021-02-28 15:00:00+00:00 - GFNS JENSIE E-FOND...,True
2021-02-28 20:00:00+00:00,GFNS JENSIE E-FONDO,02-28 20:00,/events/56540,[race],70,43.43 km,1.30 km,616 m,7. E Fondo Jensi V3 Test,/courses/107793,False,tb4vj7391iv1vlela1fif5jbv8,2021-02-28 20:00:00+00:00 - GFNS JENSIE E-FOND...,True


In [59]:
df.loc[df['name'].str.startswith('Wacky Races')]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
